# Tutorial for HPS multiple A1-LCD WTARO 

In [1]:
# load packages
import numpy as np
import pandas as pd
import sys
import os
import time
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit

import mdtraj
try:
    import nglview
except ImportError:
    print('Please install nglview to visualize molecules in the jupyter notebooks.')

sys.path.append('../../')
from openabc.forcefields.parsers import HPSParser
from openabc.forcefields import HPSModel
from openabc.utils.helper_functions import build_straight_CA_chain, write_pdb
from openabc.utils.insert import insert_molecules

# set simulation platform
#platform_name = 'CPU'
platform_name = 'CUDA'
#platform_name = 'OpenCL'

Parse a single A1LCD WT. The sequence length is 137.

In [2]:
#sequence = 'GSMASASSSQRGRSGSGNFGGGRGGGFGGNDNFGRGGNFSGRGGFGGSRGGGGYGGSGDGYNGFGNDGSNFGGGGSYNDFGNYNNQSSNFGPMKGGNFGGRSSGGSGGGGQYFAKPRNQGGYGGSSSSSSYGSGRRF' # WT
sequence = 'GSMASASSSQRGRSGSGNSGGGRGGGFGGNDNFGRGGNSSGRGGFGGSRGGGGYGGSGDGYNGFGNDGSNSGGGGSSNDFGNYNNQSSNFGPMKGGNFGGRSSGGSGGGGQYSAKPRNQGGYGGSSSSSSSGSGRRF' # ARO-
ca_pdb = 'init_A1LCD_WT_CA.pdb'
ca_atoms = build_straight_CA_chain(sequence, r0=0.38)
write_pdb(ca_atoms, ca_pdb)
protein_parser = HPSParser(ca_pdb)

Parse molecule with default settings.


Prepare the initial configuration.

In [3]:
# insert molecules into the simulation box randomly
n_mol = 500
if not os.path.exists('start.pdb'):
    insert_molecules(ca_pdb, 'start.pdb', n_mol, box=[100, 100, 100])
else:
    # delete and create a new start.pdb
    # otherwise, there will be an error for init_coord in the next step
    os.remove('start.pdb')
    insert_molecules(ca_pdb, 'start.pdb', n_mol, box=[100, 100, 100])


Successfully inserted 500 molecules.


Use the Urry scale optimal parameter ($\mu=1$ and $\Delta=0.08$) and run the simulation. 

In [4]:
protein = HPSModel()
for i in range(n_mol):
    protein.append_mol(protein_parser)
top = app.PDBFile('start.pdb').getTopology()
init_coord = app.PDBFile('start.pdb').getPositions()
print('size of init_coord:', len(init_coord))
protein.create_system(top, box_a=100, box_b=100, box_c=100)
protein.add_protein_bonds(force_group=1)
protein.add_contacts('Urry', mu=1, delta=0.08, force_group=2)
protein.add_dh_elec(force_group=3)
temperature = 300*unit.kelvin
friction_coeff = 0.01/unit.picosecond # use smaller friction coefficient to accelerate dynamics
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
protein.set_simulation(integrator, platform_name, init_coord=init_coord)
protein.simulation.minimizeEnergy()
output_interval = 1000
output_dcd = 'output_multi_A1LCD_WT.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.setVelocitiesToTemperature(temperature)

# run simulation
start = time.time()
totaL_steps = 100000
protein.simulation.step(totaL_steps)
end = time.time()
print('Total time for %d steps: %.2f s' % (totaL_steps, end-start))

size of init_coord: 68500
Add protein bonds.
Add nonbonded contacts.
Use Urry hydropathy scale.
Scale factor mu = 1 and shift delta = 0.08.
Add Debye-Huckel electrostatic interactions.
Set Debye length as 1 nm.
Set water dielectric as 80.0.
Use platform: CUDA
Use precision: mixed
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
1000,9.999999999999831,38089.32484807741,219834.10569662144,257923.43054469884,257.32702288446274,0
2000,20.000000000000327,29111.324017898212,234224.9038362165,263336.2278541147,274.17218542399934,2.52e+03
3000,30.00000000000189,27612.305279882992,239267.8399939419,266880.14527382486,280.0751991713509,2.49e+03
4000,40.00000000000061,27385.00650107625,242184.6555011535,269569.6620022297,283.48948035577195,2.44e+03
5000,49.99999999999862,27197.761921323916,244917.4112050528,272115.1731263767,286.6883101612135,2.42e+03
6000,59.99999999999663,28079.956428820962,246246.5326587224,

In [5]:
# visualize trajectory
traj = mdtraj.load_dcd('output_multi_A1LCD_WT.dcd', top='start.pdb')
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True) # realign to the origin
view = nglview.show_mdtraj(traj)
view

NGLWidget(max_frame=99)

Try to perform NPT simulations at the same temperature

In [6]:
pressure = 1*unit.bar
protein.system.addForce(mm.MonteCarloBarostat(pressure, temperature))
output_interval = 1000
output_dcd = 'output_multi_A1LCD_WT_NPT.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.reinitialize(preserveState=True)
protein.simulation.step(200000)

101000,1009.9999999992265,52396.7592078893,254604.95387825457,307001.7130861439,298.02807251183543,2.13e+03
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
101000,1009.9999999992265,52396.7592078893,254604.95387825457,307001.7130861439,298.02807251183543,0
102000,1019.9999999992174,51862.37006700407,255554.17507851627,307416.54514552036,299.1391842965521,2.12e+03
102000,1019.9999999992174,51862.37006700407,255554.17507851627,307416.54514552036,299.1391842965521,1.72e+03
103000,1029.9999999992083,51737.14837602787,255416.53321872395,307153.68159475183,298.9780674857989,2.12e+03
103000,1029.9999999992083,51737.14837602787,255416.53321872395,307153.68159475183,298.9780674857989,1.69e+03
104000,1039.9999999991992,51339.60820964641,256689.38497920704,308028.99318885343,300.4680053326149,2.11e+03
104000,1039.9999999991992,51339.60820964641,256689.38497920704,308028.99318885343,300.4680053326149,1.67e+03
1

In [7]:
# visualize trajectory
traj = mdtraj.load_dcd('output_multi_A1LCD_WT_NPT.dcd', top='start.pdb')
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True) # realign to the origin
view = nglview.show_mdtraj(traj)
view

NGLWidget(max_frame=199)